In [1]:
import pandas as pd
import numpy as np
import random
import datetime

https://leetcode.com/problems/students-and-examinations/?lang=pythondata

In [2]:
data = [[1, 'Alice'], [2, 'Bob'], [13, 'John'], [6, 'Alex']]
students = pd.DataFrame(data, columns=['student_id', 'student_name']).astype({'student_id':'Int64', 'student_name':'object'})
data = [['Math'], ['Physics'], ['Programming']]
subjects = pd.DataFrame(data, columns=['subject_name']).astype({'subject_name':'object'})
data = [[1, 'Math'], [1, 'Physics'], [1, 'Programming'], [2, 'Programming'], [1, 'Physics'], [1, 'Math'], [13, 'Math'], [13, 'Programming'], [13, 'Physics'], [2, 'Math'], [1, 'Math']]
examinations = pd.DataFrame(data, columns=['student_id', 'subject_name']).astype({'student_id':'Int64', 'subject_name':'object'})

In [3]:
students

,student_id,student_name
0,1,Alice
1,2,Bob
2,13,John
3,6,Alex


In [35]:
subjects

,subject_name
0,Math
1,Physics
2,Programming


In [5]:
examinations

,student_id,subject_name
0,1,Math
1,1,Physics
2,1,Programming
3,2,Programming
4,1,Physics
5,1,Math
6,13,Math
7,13,Programming
8,13,Physics
9,2,Math


In [33]:
df = students.merge(examinations, on='student_id')
df

,student_id,student_name,subject_name
0,1,Alice,Math
1,1,Alice,Physics
2,1,Alice,Programming
3,1,Alice,Physics
4,1,Alice,Math
5,1,Alice,Math
6,2,Bob,Programming
7,2,Bob,Math
8,13,John,Math
9,13,John,Programming


In [80]:
# 두 DataFrame 간에 겹치는 column이 없으면 how를 left 즉, students DataFrame 기준으로 합치려고 해도 안 합쳐진다.
ddf = students.merge(subjects, how='left')
ddf

MergeError: ignored

In [81]:
ddf = students.merge(subjects, how='cross')
ddf

,student_id,student_name,subject_name
0,1,Alice,Math
1,1,Alice,Physics
2,1,Alice,Programming
3,2,Bob,Math
4,2,Bob,Physics
5,2,Bob,Programming
6,13,John,Math
7,13,John,Physics
8,13,John,Programming
9,6,Alex,Math


In [71]:
# agg함수를 이런 식으로 새로운 column을 추가하는 방식으로도 쓸 수 있다...
exam_count=examinations.groupby(['student_id','subject_name'], as_index=False).agg(attended_exams=('subject_name','count'))
exam_count

,student_id,subject_name,attended_exams
0,1,Math,3
1,1,Physics,2
2,1,Programming,1
3,2,Math,1
4,2,Programming,1
5,13,Math,1
6,13,Physics,1
7,13,Programming,1


In [72]:
output = ddf.merge(exam_count, on=['student_id', 'subject_name']).sort_values(by=['student_id','subject_name']).fillna(0)
output

,student_id,student_name,subject_name,attended_exams
0,1,Alice,Math,3
1,1,Alice,Physics,2
2,1,Alice,Programming,1
3,2,Bob,Math,1
4,2,Bob,Programming,1
5,13,John,Math,1
6,13,John,Physics,1
7,13,John,Programming,1


In [38]:
df[ (df['student_name']=='Alex') & (df['subject_name']=='Math') ]

,student_id,student_name,subject_name


In [37]:
df[ (df['student_name']=='Alex') & (df['subject_name']=='Math') ].groupby('subject_name', as_index=False).agg({'student_name':'count'})['student_name'].empty

True

In [41]:
# 비어있는 Series가 반환되면 이런 에러가 뜬다. 비어있으니 0번째 인덱스가 없다는 소리인 것 같다.
lst=[]
lst.append(df[ (df['student_name']=='Bob') & (df['subject_name']=='Physics') ].groupby('subject_name', as_index=False).agg({'student_name':'count'})['student_name'][0])
lst

KeyError: ignored

In [45]:
np.sort(students['student_id'].unique())

array([1, 2, 6, 13], dtype=object)

In [47]:
num = []
for student_id in np.sort(students['student_id'].unique()):
  print(student_id)
  for subject_name in subjects['subject_name'].unique():
    print(subject_name)
    count_val = df[ (df['student_id']==student_id) & (df['subject_name']==subject_name) ].groupby('subject_name', as_index=False).agg({'student_id':'count'})['student_id']
    if count_val.empty:
      num.append(0)
    else:
      num.append(count_val[0])
    print(num)
print(f'final result : {num}')

1
Math
[3]
Physics
[3, 2]
Programming
[3, 2, 1]
2
Math
[3, 2, 1, 1]
Physics
[3, 2, 1, 1, 0]
Programming
[3, 2, 1, 1, 0, 1]
6
Math
[3, 2, 1, 1, 0, 1, 0]
Physics
[3, 2, 1, 1, 0, 1, 0, 0]
Programming
[3, 2, 1, 1, 0, 1, 0, 0, 0]
13
Math
[3, 2, 1, 1, 0, 1, 0, 0, 0, 1]
Physics
[3, 2, 1, 1, 0, 1, 0, 0, 0, 1, 1]
Programming
[3, 2, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1]
final result : [3, 2, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1]


In [42]:
def students_and_examinations(students: pd.DataFrame, subjects: pd.DataFrame, examinations: pd.DataFrame) -> pd.DataFrame:
    df = students.merge(subjects, how='cross') # how를 'cross'로 해놓음으로써 cartesian product를 실행한다.

    exam_count = examinations.groupby(['student_id', 'subject_name'], as_index=False).agg(attended_exams=('subject_name', 'count'))

    output = df.merge(exam_count, on=['student_id','subject_name'], how='left').sort_values(by=['student_id', 'subject_name']).fillna(0)[['student_id','student_name','subject_name','attended_exams']]

    return output


""
